Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [4]:
from collections import Counter
import numpy as np

In [5]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [6]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

## Montecarlo Evaluation

In [7]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

## Example

In [12]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1)
play(board, 3, -1)
# play(board, 4, 1)
# play(board, 0, -1)
# play(board, 5, 1)
print(board)
#eval_board(board, 1)

[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


In [1]:
def minmax(player, board, max_step, step, max_play):
    # TREE POST-ORDER TRAVERSAL
    # player_maxer = 1
    # player_minimer = -1

    if step == max_step:
        return eval_board(board, player)

    games = dict()
    if player == 1:
        for i in range(max_play):
            play(board, i, player)
            games[minmax(False, board, max_step, step+1, max_play)] = board
        
        return max(games)
    
    else:
        for i in range(max_play):
            play(board, i, player)
            games[minmax(True, board, max_step, step+1, max_play)] = board
        
        return min(games)




In [50]:
from random import randint, seed 
from time import time

# MATCH SETTINGS
seed(time())
computer_moves_ahead = 20
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
counter = 1
chosen_col = -1

print("Connect 4 game:\n\tCOMPUTER VS HUMAN")
print("FIGHT")
player = randint(-1, 1)
print("Human player start") if player is 1 else print("Computer start")


while True:
    print(f"\nTurn number{counter}")
    counter += 1
    print(board)

    if player==1:
        print("HUMAN")
        print("Chose a column:")
        input(chosen_col)
        play(board, int(chosen_col), player)
        print(board)

    else:
        print("COMPUTER")
        evaluation, board = minmax(player, board.copy(), computer_moves_ahead, 0, NUM_COLUMNS)
        print(f"Evaluation of the plays: {evaluation}")




Connect 4 game:
	COMPUTER VS HUMAN
FIGHT
Computer start

Turn number1
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
COMPUTER


TypeError: play() missing 2 required positional arguments: 'column' and 'player'

In [10]:
import numpy as np

vec = np.array([[ 0,  0 , 0 , 0,  0 , 0],
[ 0 , 0 , 0 , 0 , 0 , 0],
[-1 , 1 , 1 , 0,  0 , 0],
[-1 , 1, -1 , 0,  0 , 0],
[ 1 , 0 , 0,  0  ,0,  0],
[-1 ,-1 ,-1 , 1,  0  ,0],
[ 1 , 0 , 0 , 0,  0,  0]])
 
eval_table = np.array([
                            [3,4,5,5,4,3],
                            [4,6,8,8,6,4],
                            [5,8,11,11,8,5],
                            [7,10,13,13,10,7],
                            [5,8,11,11,8,5],
                            [4,6,8,8,6,4],
                            [3,4,5,5,4,3]
                            ])
print(eval_table.sub())

AttributeError: 'numpy.ndarray' object has no attribute 'sub'